In [30]:
!mkdir ./databases/DB_Info/CVP/

In [31]:
!grep ^">" ./databases/CVP.fa > ./databases/DB_Info/CVP/CVP_titles.txt

In [64]:
cvp=open("./databases/DB_Info/CVP/CVP_titles.txt").readlines()

cvp_dict={}

headers=[]

for line in cvp:
    defs={}
    annotation=""
    info=line.split("/")[1:]
    ID=line.split("/")[0]
    head=line[1:5]
    for i in info: 
        if "=" in i:
            defs[i.split("=")[0]]=i.split("=")[1]
    if "DESCRIPTION" in defs.keys():
        annotation=defs["DESCRIPTION"]
    elif "definition" in defs.keys():
        annotation=defs["definition"]
    if head not in headers:
        headers.append(head)
        print(line)
    cvp_dict[ID]=annotation


>NCBI_PEP_371783045 /NCBI_ACCESSION=CCE57270.1 /GI=371783045 /TAXON_ID=10366 /DESCRIPTION="e1 protein" /LENGTH=329

>RF_PEP_377830075 /NCBI_ACCESSION=YP_005296290.1 /GI=377830075 /TAXON_ID=930275 /DESCRIPTION="unnamed protein product [Cotia virus SPAn232]." /LENGTH=324

>SP_PEP_P19385 /ACCESSION=P19385 /DNA_SOURCE=LYS_BPCP7 /ORGANISM="Streptococcus phage CP-7" /TAXON_ID=10748 /DESCRIPTION="Lysozyme (EC 3.2.1.17) (Endolysin) (Muramidase) (CP-7 lysin)." /LENGTH=342 /ASSEMBLY_STATUS= 

>>RF_PEP_124484623/ACCESSION=YP_001031226.1/GI=124484623/DNA_SOURCE=NC_008949/ORGANISM="Hyposoter fugitivus ichnovirus"/TAXON_ID=265522/DESCRIPTION="innexin Vnx-b7"/LENGTH=376/KEYWORDS="assembly_status:finished, data_type:peptide, taxon_group:Viruses, project:"

>OMNI_PEP_KVP40_0136 /ACCESSION=KVP40_0136 /GI= /DNA_SOURCE=KVP40_0136 /ORGANISM="Vibrio phage KVP40" /TAXON_ID=75320 /DESCRIPTION="cytochrome-c oxidase, subunit 2, truncation" /LENGTH=178 /ASSEMBLY_STATUS= 

>CAMPEP_0000021618 /definition="hypothet

In [2]:
%%bash
mkdir ./databases/DB_Info/kegg
grep ^">" ./databases/kegg.reduced.fasta > ./databases/DB_Info/kegg/kegg.reduced.titles.txt

In [51]:
import re

k_func_dict={}
kegg_grp_dict={}
kegg_dict={}

kegg=open("./databases/DB_Info/kegg/kegg.reduced.titles.txt").readlines()
for k in kegg:
    k= k.replace("\n","").replace(">","").split(" ")
    name=k[0]
    info=" ".join(k[1:len(k)]).split(";")
    function=info[0]
    
    if len(info)>1:
        kegginfo=info[1].split(" ")
        if len(kegginfo)>1 and kegginfo[1].startswith("K"):
            K_group=kegginfo[1]
            kfunc=" ".join(kegginfo[2:len(kegginfo)])
            if K_group not in k_func_dict.keys():
                k_func_dict[K_group]=kfunc
                kegg_dict[name]=kfunc
                kegg_grp_dict[name]=K_group
            else:
                kegg_dict[name]=function
        else:
            kegg_dict[name]=function
    else:
        kegg_dict[name]=function

In [52]:
print len(kegg)
print (len(kegg_dict.keys()))

1291309
1291307


In [34]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/annotation/blasts/*/1.161.O.* ./

1.161.O.vs.aclame.out                         100%   32KB  31.6KB/s   00:00    
1.161.O.vs.cogs_2003-2014.out                 100%  271KB 270.7KB/s   00:00    
1.161.O.vs.CVP.out                            100%  260KB 259.9KB/s   00:00    
1.161.O.vs.kegg.out                           100%  190KB 189.7KB/s   00:00    
1.161.O.vs.Pfam.out                           100%  771KB 771.3KB/s   00:00    


In [53]:
def get_digits(prodfile):
    prod=open(prodfile).readlines()
    digits=len(str(len(prod)/2))
    return digits


def get_locus_tag(line, digits, phage):
    query=line.split("\t")[0].split(" ")[0]
    number=query.split("_")[-1]
    z="0"*(digits-len(number))
    return "NVP"+phage.replace(".","")+"_"+z+number

In [60]:
blast=open("1.161.O.vs.kegg.out").readlines()
digits=get_digits("1.161.O.gene")
phage="1.161.O."

kegg_blast_dict={}

for line in blast:

    locus_tag=get_locus_tag(line, digits, phage)
    hit=line.split("\t")[1]
    e=line.split("\t")[-2]
    bitscore=line.split("\t")[-1].replace("\n","")
    kfunc=kegg_dict.get(hit,"")
    kgroup=kegg_grp_dict.get(hit,"")
    info=[hit, e, bitscore, kfunc]
    if locus_tag not in kegg_blast_dict.keys():
        kegg_blast_dict[locus_tag]=info
        print(kfunc+"\t"+kgroup)

dcm	
CRISPR-associated helicase Cas3	
sin	
dihydrofolate reductase (EC:1.5.1.3)	
nrdF	
DNA-methyltransferase	
putative Clp protease, phage associated	
thyA	
ribonucleotide-diphosphate reductase subunit alpha (EC:1.17.4.1)	
rdgC	
prfB	
DnaB domain-containing protein	
